# CS5446 Warehouse Robot (Modular Helpers)
使用拆分后的 py 模块，让 Notebook 只专注于实验流程。

In [0]:
from rddl_utils import (
    summarize_file, pretty_domain, pretty_instance, pretty_nonfluents, print_state_action_overview
)
from grid_planning import (
    build_grid, plot_grid, astar, path_to_actions, plot_path,
    single_agent_demo, multi_agent_demo, auto_pick_starts_goals,
    multi_agent_sequential, plot_multiple_paths_pretty_offset, evaluate_multi_agent
)
from pickplace_utils import (
    parse_pick_drop_from_nonfluent_text, parse_start_from_instance_text, validate_or_fallback,
    plan_pick_and_drop, plot_pickdrop_offset, animate_pickdrop_gif_offset,
    plot_pickdrop_offset_smooth, animate_pickdrop_smooth_gif, astar_with_footprint,
    plot_explored_heatmap, plot_heuristic_field
)
from rddl_utils import read_text
from custom_layouts import make_enhanced_nonfl, ascii_preview


In [0]:
FILENAMES = {
    'domain': 'warehouse_domain_formal.rddl',
    'instance': 'warehouse_instance_6x6_formal.rddl',
    'nonfl_clear': 'warehouse_nonfluents_6x6_clear_formal.rddl',
    'nonfl_shelf': 'warehouse_nonfluents_6x6_shelves_formal.rddl',
}

NEW_FILES = {
    'domain': 'warehouse_pickplace_domain.rddl',
    'nonfl': 'warehouse_pickplace_nonfluents_15x10.rddl',
    'instance': 'warehouse_pickplace_instance_15x10.rddl',
}


In [0]:
print(ascii_preview())
enhanced_nonfl = make_enhanced_nonfl()
plot_grid(enhanced_nonfl, title='Enhanced 10x15 warehouse')


In [0]:
kind_dom, dom = summarize_file(FILENAMES['domain'])
kind_inst, inst = summarize_file(FILENAMES['instance'])
kind_clr, nfl_clear = summarize_file(FILENAMES['nonfl_clear'])
kind_shf, nfl_shelf = summarize_file(FILENAMES['nonfl_shelf'])

print('='*80); print('[Domain]'); print('='*80)
pretty_domain(dom)

print('='*80); print('[Instance]'); print('='*80)
pretty_instance(inst)

print('='*80); print('[Non-fluents] CLEAR'); print('='*80)
pretty_nonfluents(nfl_clear, 'CLEAR')

print('='*80); print('[Non-fluents] SHELVES'); print('='*80)
pretty_nonfluents(nfl_shelf, 'SHELVES')

print_state_action_overview(dom, nfl_clear, 'CLEAR')
print_state_action_overview(dom, nfl_shelf, 'SHELVES')


In [0]:
start_xy = inst.get('init_agent_xy')
print('Start (from instance):', start_xy)
print('Goal (CLEAR):', (nfl_clear.get('GOAL_X'), nfl_clear.get('GOAL_Y')))
print('Goal (SHELVES):', (nfl_shelf.get('GOAL_X'), nfl_shelf.get('GOAL_Y')))
plot_grid(nfl_clear, start_xy, title='CLEAR map')
plot_grid(nfl_shelf, start_xy, title='SHELVES map')


In [0]:
print('=== CLEAR map ===')
single_agent_demo(nfl_clear, inst)

print('=== SHELVES map ===')
single_agent_demo(nfl_shelf, inst)

grid_sh = build_grid(nfl_shelf)
starts, goals = auto_pick_starts_goals(nfl_shelf, k=3, seed=13)
plans = multi_agent_sequential(grid_sh, starts, goals, t_max=256)
plot_multiple_paths_pretty_offset(nfl_shelf, plans, starts=starts, goals=goals, title='Multi-agent sequential A* (SHELVES)')
metrics = evaluate_multi_agent(grid_sh, starts, goals, plans)
print('Multi-agent metrics:', metrics)


In [0]:
kind_d2, dom2 = summarize_file(NEW_FILES['domain'])
kind_nf2, nf2 = summarize_file(NEW_FILES['nonfl'])
kind_ins2, ins2 = summarize_file(NEW_FILES['instance'])

print('='*80); print('[Pick&Place Domain]'); print('='*80)
pretty_domain(dom2)

print('='*80); print('[Pick&Place Non-fluents]'); print('='*80)
pretty_nonfluents(nf2, 'Pick&Place')

print('='*80); print('[Pick&Place Instance]'); print('='*80)
pretty_instance(ins2)

print_state_action_overview(dom2, nf2, '15x10 pick&place')
plot_grid(nf2, start_xy=ins2.get('init_agent_xy'), title='Pick&Place 15x10 map')


In [0]:
nf_text = read_text(NEW_FILES['nonfl'])
ins_text = read_text(NEW_FILES['instance'])
start_xy = parse_start_from_instance_text(ins_text) or ins2.get('init_agent_xy')
start_xy = validate_or_fallback(nf2, start_xy)
pick_set, drop_set = parse_pick_drop_from_nonfluent_text(nf_text)
if not pick_set:
    pick_set = [start_xy]
if not drop_set:
    drop_set = [pick_set[0]]
pick_xy, drop_xy = pick_set[0], drop_set[0]
pick_xy = validate_or_fallback(nf2, pick_xy)
drop_xy = validate_or_fallback(nf2, drop_xy)
print(f'[Pick&Place] start={start_xy}, pick={pick_xy}, drop={drop_xy}')
p1, p2 = plan_pick_and_drop(nf2, start_xy, pick_xy, drop_xy)
print('Phase-1 length:', None if p1 is None else len(p1)-1)
print('Phase-2 length:', None if p2 is None else len(p2)-1)
plot_pickdrop_offset(nf2, p1, p2, start_xy, pick_xy, drop_xy, title='Pick&Place S→P→D (offset)')
plot_pickdrop_offset_smooth(nf2, p1, p2, start_xy, pick_xy, drop_xy, title='Pick&Place (smooth+offset)')
animate_pickdrop_gif_offset(nf2, p1, p2, start_xy, pick_xy, drop_xy, out_gif='pickplace_15x10_offset.gif')
animate_pickdrop_smooth_gif(nf2, p1, p2, start_xy, pick_xy, drop_xy, out_gif='pickplace_15x10_smooth.gif')


In [0]:
grid = build_grid(nf2)
p1_fp, fp1 = astar_with_footprint(grid, start_xy, pick_xy)
p2_fp, fp2 = astar_with_footprint(grid, pick_xy, drop_xy)
plot_explored_heatmap(nf2, fp1, title='A* explored heatmap (S→P)')
plot_explored_heatmap(nf2, fp2, title='A* explored heatmap (P→D)')
plot_heuristic_field(nf2, fp1, field='h')
plot_heuristic_field(nf2, fp1, field='f')


In [0]:
# Enhanced layout visualization with semantic annotations
enhanced_nonfl = make_enhanced_nonfl()
plot_grid(enhanced_nonfl, title='Enhanced 10x15 warehouse (annotated)', annotate_special=True, figsize=(7,7))


In [0]:
# Preview updated outbound-rich layout
print(ascii_preview())
